In [13]:
import numpy as np
import pandas as pd
from sklearn import metrics

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import transformers
from transformers import BertTokenizer, BertModel, BertConfig

In [14]:
#Setting up the device for GPU usage
device = torch.device("cuda" if torch.cuda.is_available() 
                      else  "mps" if torch.backends.mps.is_available()
                      else "cpu"
                      )

In [15]:
#Initialize a BERT tokenizer from the 'bert-base-uncased' pre-trained model.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
class AmazonTitles_Dataset(Dataset):
    """
    A PyTorch Dataset class for tokenizing and encoding Amazon product titles
    using a given tokenizer and preparing them for model training or inference.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the titles and polarity labels.
        tokenizer: The tokenizer object used to tokenize and encode the titles.
        max_len (int): The maximum length of the tokenized titles after encoding.
    """

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer=tokenizer
        self.data=dataframe
        self.titles=dataframe.title
        self.targets=self.data.polarity
        self.max.len=max_len
    
    def __len__(self):
        return len(self.titles)
    
    def __getitem__(self, index):
        titles=str(self.titles[index])
        titles = " ".join(titles.split())
    
        inputs=self.tokenizer.encode_plus(
            titles,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids=inputs['input_ids']
        mask=inputs['attention_mask']
        token_type_ids=inputs['token_type_ids'] 

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.int)
        }      

In [37]:
MAX_LEN=512
BATCH_SIZE=512
LEARNING_RATE=0.1

In [30]:
class BertClass(torch.nn.Module):

    def __init__(self, dropout=0.1):
        super(BertClass, self).__init__()
        self.transformer = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.drop1 = torch.nn.Dropout(dropout)
        self.l1 = torch.nn.Linear(768, 300)
        self.act1=torch.nn.Tanh()
        self.drop2 = torch.nn.Dropout(dropout)
        self.l2 = torch.nn.Linear(300, 100)
        self.act2=torch.nn.Tanh()
        self.drop3=torch.nn.Dropout(dropout)
        self.l3=torch.nn.Linear(100,1)
        self.out = torch.nn.Sigmoid()

    def forward(self, ids, mask, token_type_ids):
        output_1=self.transformer(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        output_2=self.drop1(output_1)
        output_3=self.drop2(self.act1(self.l1(output_2)))
        output_4=self.drop3(self.act2(self.l2(output_3)))
        output=self.out(self.l3(output_4))

        return output

In [31]:
model=BertClass()
model.to(device)

BertClass(
  (transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [40]:
def loss_fn(outputs, targets):
    return torch.nn.BCELoss()(outputs, targets)

In [38]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)